# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234973298206                   -0.50    8.0
  2   -7.250268719956       -1.82       -1.40    1.0
  3   -7.250843731184       -3.24       -1.78    4.0
  4   -7.250912588820       -4.16       -1.85    2.0
  5   -7.251331272343       -3.38       -2.65    2.0
  6   -7.251338270308       -5.16       -3.22    1.0
  7   -7.251338689780       -6.38       -3.54    3.0
  8   -7.251338793894       -6.98       -4.13    2.0
  9   -7.251338797944       -8.39       -4.67    3.0
 10   -7.251338798619       -9.17       -5.06    4.0
 11   -7.251338798691      -10.14       -5.44    2.0
 12   -7.251338798703      -10.91       -5.96    3.0
 13   -7.251338798704      -12.08       -6.28    2.0
 14   -7.251338798705      -12.58       -6.87    2.0
 15   -7.251338798705      -13.82       -7.23    3.0
 16   -7.251338798705      -14.45       -7.56    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.053854806283895405
[ Info: Arnoldi iteration step 2: normres = 0.3606119806813314
[ Info: Arnoldi iteration step 3: normres = 0.30234709902122514
[ Info: Arnoldi iteration step 4: normres = 0.9434171519126342
[ Info: Arnoldi iteration step 5: normres = 0.5629096110187453
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.25e-01, 5.07e-02, 3.74e-01, 2.61e-01, 3.67e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3879153764068761
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.05e-01, 6.18e-02, 3.29e-01, 1.32e-01, 5.66e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08247689334026745
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.48e-03, 7.29e-03, 2.55e-02, 5.48e-02, 4.80e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10544709303293347
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.03e